In [ ]:
import sys


no_of_cpu =8
max_cores =16
executor_mem = '56g'


Job_Name = 'Nishant: Cover_details'


from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import asc,lit
#warnings.filterwarnings('error')
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F

conf = pyspark.SparkConf()
#import numpy
import calendar
#import pandas as pd
#import simplejson as json
#import pandas as pd
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from pyspark.sql.functions import lit
import simplejson as json
import json, pprint, requests
#es_nodes = '10.35.12.5'
#es_nodes = '10.35.12.6'
es_nodes = '10.35.12.194'#,10.35.12.6,10.35.12.5
es_nodes_temp='10.35.12.194'
es_port = '5432'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'
mesos_ip = 'mesos://10.33.195.18:5050'#'mesos://10.35.12.5:5050'

spark.stop() #############NEED TO COMMENT THIS SPARK.STOP WHEN RUNNING THROUGH SHELL###############################

conf.setMaster(mesos_ip)

conf.set('spark.executor.cores',no_of_cpu) ### change 1
#conf.set('spark.memory.fraction','.2')
conf.set('spark.executor.memory',executor_mem) 
conf.set('spark.es.scroll.size','10000')
conf.set('spark.network.timeout','600s')
conf.set('spark.sql.crossJoin.enabled', 'true')

conf.set('spark.ui.port','4052')

conf.set('spark.executor.heartbeatInterval','60s')
conf.set("spark.driver.cores","4")
conf.set("spark.driver.extraJavaOptions","-Xmx4g -Xms4g")

#conf.set("spark.shuffle.blockTransferService", "nio"); 
conf.set("spark.files.overwrite","true");
conf.set("spark.kryoserializer.buffer", "70"); 
conf.set("spark.driver.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.executor.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer"); 
conf.set("spark.broadcast.compress", "true"); 
conf.set("spark.shuffle.compress", "true"); 
conf.set("spark.shuffle.spill.compress", "true");
conf.set("spark.app.name", Job_Name)
#conf.set("spark.io.compression.codec","org.apache.spark.io.LZ4CompressionCodec");
#conf.set("spark.sql.inMemoryColumnarStorage.compressed", "true"); 
from pyspark.sql.functions import broadcast

conf.set('spark.driver.memory','20g') ### change 2
conf.set('spark.cores.max',max_cores) ### change 3
conf.set('spark.sql.shuffle.partitions','400')
#conf.set('spark.sql.crossJoin.enabled', 'true')
conf.set('es.nodes',es_nodes)
conf.set('es.port',es_port)
conf.set('es.nodes.wan.only','true')
conf.set("spark.sql.autoBroadcastJoinThreshold",-1)

#conf.set('spark.es.net.http.auth.user','Spark')
#conf.set('spark.es.net.http.auth.pass','Jarkpet1Sap3')
conf.set('spark.num.executors','4')
conf.set('spark.debug.maxToStringFields', 200)

conf.set('spark.es.net.http.auth.user', es_user)
conf.set('spark.es.net.http.auth.pass', es_pwd)

conf.set('spark.es.mapping.date.rich','false')
spark = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(spark)

# Load Data into PySpark DataFrames
# Prodcom Data Frame
import json, pprint, requests
import pyspark.sql.functions as sf
from pyspark.sql.types import *


import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType



In [ ]:
from pyspark.sql.functions import hash,col,regexp_replace,collect_list,concat_ws,initcap,count,least
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
import psycopg2
import time
from time import gmtime, strftime
import pytz
starttime = time.time()
start_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S.%f")
start_time

In [ ]:
prod_url = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
prod_host = '10.35.12.194'
prod_port = '5432'
prod_dbname = 'gpadmin'
user_prod="gpcustcoe"
pwd_prod="gpcustcoe@123"
dbschema="public"


prod_gpdb_spark_options ={
    "url": "jdbc:postgresql://{host}:{port}/{dbname}".format(host=prod_host,port=prod_port, dbname=prod_dbname),
    "user": "{user}".format(user=user_prod),
    "password": "{password}".format(password=pwd_prod)
} 

In [ ]:
def get_latest_progress(table_name,column_name):
    
    try: 
        gscPythonOptions = {
                 "url":prod_url ,
                 "user":user_prod ,
                 "password": pwd_prod,
                 "dbschema":"customermart",
                 "dbtable": table_name,#table change
                 "server.port":"1150-1170"} 
        
        # this query will fetch till what date we have inserted the records in target
        last_run= sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
            .select(column_name)
        
        Max_last_run =last_run.select(max(column_name)).first()[0]


        # if no record is available in progress then it will raise value error, in this case default value will be picked from except clause
        if Max_last_run is None:
            raise ValueError("No records found!")

        #print("Record Found. Progress updated till {}".format(Max_last_run))

    except Exception as E: 
        Max_last_run = datetime.datetime(2018, 10, 31, 0, 0)  # this is the default start date when no record is present in progress for customer_demographics table
        print("Executed_except",repr(E))
    return Max_last_run

In [ ]:
# loading table as dataframe
def load_gpdb_jdbc(col_str,dbtable,dbschema,col_name=None,time_filter=None,partitionColumn="row_num"):
    """
    This is used to read gpdb with filter for columns and can apply other filter(date,values).
    Time filter contains startdate,enddate
    """
    gscPythonOptions = {
                        "url": prod_url,
                        "user": user_prod,
                        "password": pwd_prod,
                        "dbschema": dbschema,
                        "dbtable": dbtable,
                        "partitionColumn":partitionColumn,
                        "partitions": 8,
                        "server.port":"1150-1170"}
   
    if time_filter:
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates().filter(col(col_name).between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd')))
    else :
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates()
   
    return data

In [ ]:
gscPythonOptions = {
         "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpcustcoe","password": "gpcustcoe@123",
    "dbschema":"customermart",
    "dbtable": "header",
         "server.port":"1150-1170"}


header_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.select('golden_id','source_system_customer_id','active_flag','mapping_date').filter((col('active_flag')=='1')).filter((col('source_system_customer_id').isNotNull()))

In [ ]:
from pyspark.sql.window import Window

header_df = header_df.withColumn("rn", F.row_number()
        .over(Window.partitionBy('source_system_customer_id')
        .orderBy(F.col("mapping_date").desc(),F.col("golden_id").desc())))

header_df = header_df.filter(F.col("rn") == 1).drop("rn")

header_df=header_df.select('source_system_customer_id','golden_id','mapping_date')

In [ ]:
col_str=['golden_id','full_name','first_name','last_name','birth_date','gender','marital_status','weight_kg','tobacco_use_indicator','alcohol_use_indicator','height_cm']
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark", "server.port":"1150-1170",
"password": "spark@456","dbschema":"customermart",
"dbtable": "customer_demographics"}
demog_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)

demog_df=demog_df.withColumn('customer_name',coalesce(col('full_name'),concat(col('first_name'),lit(' '),col('last_name'))))


In [ ]:
col_str=['golden_id','service_level']
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark", "server.port":"1150-1170",
"password": "spark@456","dbschema":"customermart",
"dbtable": "customer_profile"}
profile_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)


In [ ]:
gscPythonOptions = {
         "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpcustcoe","password": "gpcustcoe@123"
     ,"dbschema":"customermart"
    ,"dbtable": "address",
         "server.port":"1150-1170"}


address_retail_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.select('source_system_customer_id','address_type','city','last_updated')\
.filter((col('address_type')=='home'))

In [ ]:
from pyspark.sql.window import Window

address_retail_df = address_retail_df.withColumn("rn", F.row_number()
        .over(Window.partitionBy('source_system_customer_id','address_type')
        .orderBy(F.col("last_updated").desc())))

address_retail_df = address_retail_df.filter(F.col("rn") == 1).select('source_system_customer_id','city')

In [ ]:
report_date_str='2023-09-01'

In [ ]:
col_str=["policy_number",'certificate_no', 'product_name', 'product_cd', 'reference_num',
         'pol_incept_date as policy_start_date', 'pol_exp_date as policy_end_date','source_system_name',
        'source_system_customer_id', 'lob', 'sum_insured', "uw_discount", 'premiumamount as premium',
        'veh_make', 'veh_model', 'veh_no as vehicle_registration_no', 'ncb_percent','producer_cd',
        'plan_type as plan_name','status','record_type_desc','commit_timestamp as binddate','veh_engine_no','veh_chassis_no',
         'times_renewed_cnt','min_pol_incept_date','no_of_lives']
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpcustcoe",
"password": "gpcustcoe@123","dbschema":"customermart","server.port":"1150-1170",
"dbtable": "policy_retail"}
policy = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)
policy.columns

In [ ]:
col_str=['txt_parent_intermediary_cd as parent_producer_cd','txt_intermediary_cd as producer_cd']

gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpcustcoe",
"password": "gpcustcoe@123","dbschema":"public","server.port":"1150-1170",
"dbtable": "reference_gc_genmst_intermediary"}
producer_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)
producer_df.columns

In [ ]:
lst_cancelled=['ACDC',
'ACDE',
'APC',
'APCP',
'APD',
'APRI',
'APU',
'DCC',
'ECDU',
'ECN',
'ECR',
'EPCE',
'ERFC',
'NCN',
'RCN']

In [ ]:
policy=policy.withColumn('record_type_desc', when((col('status').isin(lst_cancelled)),lit('CANCELLATION'))\
                           .otherwise(col('record_type_desc')))

In [ ]:
#Krishnan Badrinath
policy=policy.withColumn('assets',when(col('lob').isin('2W'),'2 Wheeler')\
                                        .when(col('lob').isin('Pvt Car'),'Car')\
                                        .when(col('lob').isin('Home'),'Home').otherwise(lit('-')))

In [ ]:
w = Window.partitionBy(['policy_number','certificate_no','policy_start_date', 'policy_end_date']).orderBy([F.col('binddate').desc(),F.col('reference_num').desc()])
policy_df = policy.withColumn('row',F.row_number().over(w))
policy_df = policy_df.filter(F.col('row')==1).drop(policy_df.row)


In [ ]:
policy_df=policy_df.withColumn('certificate_no',when(col('certificate_no').isin('0','0.0','/00000','000000','00000000'),lit('00000000')).otherwise(lit(col('certificate_no'))))\
.withColumn('policy_number',lpad(col('policy_number'),10,'0'))

In [ ]:
col_str=["risk1 as relation","reference_num",'insured_name as name','insured_dob as dob']
pdrdh_df=load_gpdb_jdbc(col_str,"policy_dh_risk_details_hlt",'public')

In [ ]:
col_str=['source','scoredate', 'score','income','dc_unified_id as golden_id']
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpcustcoe", "server.port":"1150-1170",
"password": "gpcustcoe@123","dbschema":"datamarts",
"dbtable": "credit_bureau_data"}
credit_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)
credit_df.columns

In [ ]:
col_str=["policy_no", "renewal_prop_final_buckets","segment","month"]
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpcustcoe",
"password": "gpcustcoe@123",
"server.port": "1151-1170",
"dbschema":"customermart",
"dbtable": "propensity_score_jan_23_jan_24"}
ps = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)
ps.columns

In [ ]:
ps=ps.withColumn('month',to_date(col('month'),'MMM-yy'))

from pyspark.sql.window import Window

ps = ps.withColumn("rn", F.row_number()
        .over(Window.partitionBy('policy_no')
        .orderBy(F.col("month").desc())))

ps = ps.filter(F.col("rn") == 1).drop("rn")

ps=ps.withColumn('renewal_propensity',concat(col('segment'),lit("("),col("renewal_prop_final_buckets"),lit(")")))

ps=ps.select('policy_no','renewal_propensity','month')

propensity_df=ps.withColumn('policy_no',lpad(trim(col('policy_no')),10,'0'))

In [ ]:
propensity_df=propensity_df.selectExpr('policy_no as policy_number', 'renewal_propensity')

In [ ]:
col_str=["dc_unified_id as golden_id","'Eligible for cross sell to Health' as is_eligible"]
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpcustcoe",
"password": "gpcustcoe@123",
 "server.port":"1150-1170",
"dbschema":"customermart",
"dbtable": "cross_sell_eligible_cust"}
cross_sell_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)
cross_sell_df.columns

In [ ]:
gscPythonOptions = {
         "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpcustcoe","password": "gpcustcoe@123",
    "dbschema":"customermart"
    ,"dbtable": "contactability",
         "server.port":"1150-1170"}

contactibilty_retail_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.selectExpr('source_system_customer_id','communication_sub_type','value','last_updated','source_system_name','verification_flag').filter((col('source_system_customer_id').isNotNull()))\
.filter(col('value').isNotNull()).filter(col('source_system_name')!='CREDIT_BUREAU')

In [ ]:
contactibilty_retail_df=contactibilty_retail_df.withColumn("rnk",when(col("source_system_name")=='GC',1)\
.otherwise(2))

In [ ]:
contactibilty_retail_df=contactibilty_retail_df.withColumn('mobile',when(col('communication_sub_type')=='mobile',col('value')).otherwise(None))
contactibilty_retail_df=contactibilty_retail_df.withColumn('email',when(col('communication_sub_type')=='email',col('value')).otherwise(None))

contactibilty_retail_df = contactibilty_retail_df.withColumn('email',when((lower(trim(col("email"))).rlike("^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}(\.[a-zA-Z]{2,})?$") == False)\
                                                        & (lower(trim(col("communication_sub_type")))=='email'),lit(None)).otherwise(col('email')))



contactibilty_retail_df=contactibilty_retail_df.select('source_system_customer_id','mobile','email','rnk','last_updated','verification_flag')

In [ ]:
contactibilty_retail_df.columns

In [ ]:
contactibilty_retail_df=contactibilty_retail_df.join(header_df,'source_system_customer_id','inner')

In [ ]:
contactibilty_retail_df_email=contactibilty_retail_df.filter((col('email').isNotNull())).dropDuplicates(subset=["email","golden_id"])
contactibilty_retail_df_mob=contactibilty_retail_df.filter((col('mobile').isNotNull())).drop_duplicates(subset=["mobile","golden_id"])

In [ ]:
contactibilty_retail_df_email=contactibilty_retail_df_email.withColumn('email',lower(col('email')))

In [ ]:
# from pyspark.sql.window import Window

# contactibilty_retail_df_email = contactibilty_retail_df_email.withColumn("rn", F.row_number()
#         .over(Window.partitionBy('golden_id','email')
#         .orderBy(F.col("rnk").asc(),F.col("last_updated").desc(),F.col("email").desc_nulls_last())))

# contactibilty_retail_df_email = contactibilty_retail_df_email.filter(F.col("rn")<=2)

from pyspark.sql.window import Window

contactibilty_retail_df_email = contactibilty_retail_df_email.withColumn("rn", F.dense_rank()
        .over(Window.partitionBy('golden_id')
        .orderBy(F.col("rnk").asc(),F.col("last_updated").desc(),F.col("email").desc_nulls_last())))

contactibilty_retail_df_email = contactibilty_retail_df_email.filter(F.col("rn")<=2)

In [ ]:
# from pyspark.sql.window import Window

# contactibilty_retail_df_mob = contactibilty_retail_df_mob.withColumn("rn_mob", F.row_number()
#         .over(Window.partitionBy('golden_id','mobile')
#         .orderBy(F.col("rnk").asc(),F.col("last_updated").desc(),F.col("mobile").desc_nulls_last())))

# contactibilty_retail_df_mob = contactibilty_retail_df_mob.filter(F.col("rn_mob")<=2)

from pyspark.sql.window import Window

contactibilty_retail_df_mob = contactibilty_retail_df_mob.withColumn("rn_mob", F.dense_rank()
        .over(Window.partitionBy('golden_id')
        .orderBy(F.col("rnk").asc(),F.col("last_updated").desc(),F.col("mobile").desc_nulls_last())))

contactibilty_retail_df_mob = contactibilty_retail_df_mob.filter(F.col("rn_mob")<=2)

In [ ]:
contactibilty_retail_df_email=contactibilty_retail_df_email.select('golden_id','email','rn','verification_flag').filter((col('email').isNotNull())).drop_duplicates()
contactibilty_retail_df_mob=contactibilty_retail_df_mob.selectExpr('golden_id','mobile','rn_mob','verification_flag as verification_flag_mob').filter((col('mobile').isNotNull())).drop_duplicates()

In [ ]:
contactibilty_retail_df=contactibilty_retail_df_mob.join(contactibilty_retail_df_email,'golden_id','full_outer').drop_duplicates()

In [ ]:
contactibilty_retail_df.columns

In [ ]:
contactibilty_retail_df=contactibilty_retail_df.repartition('golden_id')

In [ ]:
col_str=['claim_no','claim_feature_concat','insured_name','claim_status','paid_amt_ind as claim_net',
         'paid_date_ind as claim_closed_date'
         ,'clm_create_date as claim_date','loss_reported_date as claim_reported_date',
'policy_no as policy_number','certificate_no',"pol_incept_date as policy_start_date",
         "pol_exp_date as policy_end_date",
         'claimant_name','loss_desc_txt as loss_description','customer_id as source_system_customer_id',
'coverage_desc as coverage_type','proposal_no as num_reference_no']

gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpcustcoe",
"password": "gpcustcoe@123",
"dbschema":"datamarts",
"dbtable": "producer_policy_product_wise_base_data",
"server.port": "1151-1171"}
crgp_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load().selectExpr(col_str)

#df_crp=df_crp.groupby('num_reference_no').agg(countDistinct(col('claim_no')).alias('claim_cnt'),sum(col('claim_gross').cast('double')).alias('claim_gross'))

crgp_df.columns

In [ ]:
crgp_df=crgp_df.withColumn('certificate_no',when(col('certificate_no').isin('0','0.0','/00000','000000','00000000'),lit('00000000')).otherwise(lit(col('certificate_no'))))\
.withColumn('policy_number',lpad(col('policy_number'),10,'0'))

crgp_df=crgp_df.withColumn('policy_start_date',to_date(col('policy_start_date'),'dd-MM-yyyy'))\
.withColumn('policy_end_date',to_date(col('policy_end_date'),'dd-MM-yyyy'))

In [ ]:
crgp_df=crgp_df.withColumn('policy_start_date',to_date(col('policy_start_date'),'dd-MM-yyyy'))\
.withColumn('policy_end_date',to_date(col('policy_end_date'),'dd-MM-yyyy'))\
.withColumn('claim_date',to_date(col('claim_date'),'dd-MM-yyyy'))


In [ ]:
col_str=["nameofpreexistingdisease as ped","num_reference_number as reference_num"]
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpcustcoe",
"password": "gpcustcoe@123",
"server.port": "1151-1170",
"dbschema":"public",
"dbtable": "underwriting_dh_cnfgtr_otherdt_grid_tab_hlt"}
pd_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)
pd_df.columns

In [ ]:
col_str=['feedback_title','feedback_date as date','score','golden_id']
# "current_year"
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpcustcoe",
"password": "gpcustcoe@123",
"server.port": "1151-1170",
"dbschema":"customermart",
"dbtable": "feedback_spa_v2"}
feedback_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)
feedback_df.columns

## Active

In [ ]:
pdf=policy_df.select('source_system_customer_id','policy_end_date','record_type_desc','policy_number','certificate_no')

is_policy=header_df.join(pdf,'source_system_customer_id','inner')

In [ ]:
header_df=is_policy.select('golden_id','source_system_customer_id','mapping_date')

In [ ]:
is_policy.columns

In [ ]:
is_active_df=is_policy.withColumn('cust_status', when((col('policy_end_date') >=current_date()) &(col('record_type_desc')!='CANCELLATION') , lit(1)).otherwise(lit(0)))\

In [ ]:
is_cust_active=is_active_df.groupBy('golden_id').agg(max('cust_status').alias("is_active"))


In [ ]:
is_cust_active=is_cust_active.withColumn('is_active',when(is_cust_active['is_active']==1,True).otherwise(False))

# policyDetails

In [ ]:
policy_details=header_df.join(policy_df,'source_system_customer_id',"inner").drop_duplicates()

In [ ]:
policy_details=policy_details.join(producer_df,'producer_cd',"left").drop_duplicates()

In [ ]:
from pyspark.sql.window import Window

w = Window.partitionBy(['golden_id','policy_number','certificate_no','policy_start_date', 'policy_end_date']).orderBy([F.col('binddate').desc(),F.col('reference_num').desc()])
policy_details = policy_details.withColumn('row',F.row_number().over(w))
policy_details = policy_details.filter(F.col('row')==1).drop(policy_details.row).drop_duplicates()

In [ ]:
crgp_df=crgp_df.drop_duplicates()

In [ ]:
from pyspark.sql.functions import sum,avg,max
group_cols = ["policy_number", "certificate_no",'policy_start_date', 'policy_end_date']
df_p=crgp_df.groupBy(group_cols).agg(countDistinct("claim_no").alias("claim_count"),sum("claim_net").alias("total_claim_amount"))

In [ ]:
policy_details=policy_details.join(df_p,group_cols,'left')

In [ ]:
group_cols = ["policy_number", "certificate_no",'policy_start_date', 'policy_end_date']

In [ ]:
policy_details=policy_details.join(crgp_df,group_cols,"left_outer").drop(crgp_df.num_reference_no)\
.drop(crgp_df.policy_number).drop(crgp_df.certificate_no).drop(crgp_df.source_system_customer_id).drop(crgp_df.policy_start_date)\
.drop(crgp_df.policy_end_date)

In [ ]:
policy_details_main=policy_details\
.withColumn('claim_count',when(col('claim_count').isNull(),lit('0')).otherwise(col('claim_count')))\
.withColumn('total_claim_amount',when(col('total_claim_amount').isNull(),lit('0.0')).otherwise(col('total_claim_amount')))\
.withColumn('uw_discount',when(col('uw_discount').isNull(),lit('0.0')).otherwise(col('uw_discount')))\
.withColumn('claim_no',when(col('claim_no').isNull(),lit('-')).otherwise(col('claim_no')))\
.withColumn('loss_description',when(col('loss_description').isNull(),lit('-')).otherwise(col('loss_description')))\
.withColumn('coverage_type',when(col('coverage_type').isNull(),lit('-')).otherwise(col('coverage_type')))\
.withColumn('claim_net',when(col('claim_net').isNull(),lit(None)).otherwise(col('claim_net')))\
.withColumn('total_claim_amount',when(col('total_claim_amount').isNull(),lit('0.0')).otherwise(col('total_claim_amount')))\
.withColumn('claim_closed_date',when(col('claim_closed_date').isNull(),lit(None)).otherwise(col('claim_closed_date')))\


In [ ]:
policy_details_main=policy_details_main.drop_duplicates()

In [ ]:
policy_details_main=policy_details_main.withColumn('current_year', when(year(col('policy_start_date'))==year(current_date()), lit(True)).otherwise(lit(False)))\
.withColumn('status', when(col('policy_end_date') >=current_date(), lit('Active'))\
            .when(months_between(current_date(),col('policy_end_date'))<=3,lit('DueForRenewal')).otherwise(lit('Inactive')))\
.withColumn('status_enum', when(col('policy_end_date') >=current_date(), lit(1))\
            .when(col("status")=='DueForRenewal',lit(2)).otherwise(lit(0)))

In [ ]:
policy_details_main=policy_details_main.withColumn('status', when((col('record_type_desc').isin('CANCELLATION')),lit('Inactive'))\
                           .otherwise(col('status')))\
.withColumn('status_enum', when((col('record_type_desc').isin('CANCELLATION')),lit(0))\
                           .otherwise(col('status_enum')))

In [ ]:
policy_details_main=policy_details_main.withColumn('primary_key', concat_ws('',trim(col('golden_id')),trim(col('policy_number')), trim(col('certificate_no')),trim(col('claim_feature_concat')),trim(col('policy_start_date')), trim(col('policy_end_date'))))

In [ ]:
from pyspark.sql.window import Window

w = Window.partitionBy('primary_key').orderBy([F.col('binddate').desc(),F.col('reference_num').desc()])
policy_details_main = policy_details_main.withColumn('row',F.row_number().over(w))
policy_details_main = policy_details_main.filter(F.col('row')==1).drop(policy_details_main.row).drop_duplicates()

# policyDetailsSummary

In [ ]:
policy_summary=policy_details_main.selectExpr('golden_id',"policy_number as policy_no","producer_cd as producer_code",'null as parent_producer_code',"certificate_no","lob","status","status_enum","product_name","product_cd as product_code","policy_start_date",
"policy_end_date",'sum_insured',"uw_discount as discount","premium","claim_count","total_claim_amount",'binddate','reference_num')

In [ ]:
policy_summary=policy_summary.withColumn('discount',when(col('discount').isNull(),lit('0.0')).otherwise(col('discount')))\
.withColumn('claim_count',when(col('claim_count').isNull(),lit('0')).otherwise(col('claim_count')))\
.withColumn('total_claim_amount',when(col('total_claim_amount').isNull(),lit('0.0')).otherwise(col('total_claim_amount')))

In [ ]:
policy_summary=policy_summary.withColumn('policy_start_date',date_format(to_utc_timestamp(col('policy_start_date'),"UTC"),'yyyy-MM-dd'))\
.withColumn('policy_end_date',date_format(to_utc_timestamp(col('policy_end_date'),"UTC"),'yyyy-MM-dd'))

In [ ]:
policy_summary=policy_summary.drop_duplicates()

In [ ]:
from pyspark.sql.window import Window

w = Window.partitionBy(['golden_id','policy_no','certificate_no']).orderBy([F.col('binddate').desc(),F.col('reference_num').desc()])
policy_summary = policy_summary.withColumn('row',F.row_number().over(w))
policy_summary = policy_summary.filter(F.col('row')==1).drop(policy_summary.row).drop_duplicates()

In [ ]:
policy_summary=policy_summary.withColumn('primary_key', concat_ws('',trim(col('golden_id')),trim(col('policy_no')), trim(col('certificate_no'))))

In [ ]:
from pyspark.sql.window import Window

w = Window.partitionBy('primary_key').orderBy([F.col('binddate').desc(),F.col('reference_num').desc()])
policy_summary = policy_summary.withColumn('row',F.row_number().over(w))
policy_summary = policy_summary.filter(F.col('row')==1).drop(policy_summary.row).drop_duplicates()

In [ ]:
policy_summary=policy_summary.selectExpr('golden_id',"policy_no","producer_code",'parent_producer_code',"certificate_no","lob","status","status_enum","product_name","product_code","policy_start_date",
"policy_end_date","sum_insured","discount","premium","claim_count","total_claim_amount",'binddate','primary_key')

# Lifetime_Details

In [ ]:
lifetime_df=policy_df.filter(col('record_type_desc')!='CANCELLATION')
lifetime_df=lifetime_df.join(header_df,'source_system_customer_id','inner')

In [ ]:
premium_temp=lifetime_df.select('golden_id','source_system_customer_id','premium').drop_duplicates()

In [ ]:
premium_temp=premium_temp.groupBy('golden_id').agg(sum('premium').alias("premium_received"))

In [ ]:
times_renewed=lifetime_df.groupBy('golden_id','policy_number','certificate_no').agg(max('times_renewed_cnt').alias("times_renewed")).drop_duplicates()
tenure_df=lifetime_df.groupBy('golden_id').agg(max('policy_end_date').alias('maxi_policy_end_date'),min(col('min_pol_incept_date')).alias('min_pol_incept_date')).drop_duplicates()

In [ ]:
tenure_df=tenure_df.withColumn('max_policy_end_date',least(col('maxi_policy_end_date'),current_date()))
tenure_df=tenure_df.groupBy('golden_id').agg(datediff(max('max_policy_end_date'),min('min_pol_incept_date')).alias("tenure"))

In [ ]:
# tenure_df=tenure_df.groupBy('golden_id').agg(datediff(max('max_policy_end_date'),min('min_pol_incept_date')).alias("tenure"))

In [ ]:
life_deatils=policy_details_main.select('golden_id','status','reference_num','claim_count','claim_no','total_claim_amount','policy_number', 'certificate_no','policy_end_date')\
.join(lifetime_df.select('reference_num'),'reference_num','inner')

In [ ]:
life_deatils=life_deatils.select('golden_id', 'status', 'claim_no', 'total_claim_amount', 'policy_number', 'certificate_no', 'policy_end_date').drop_duplicates()

In [ ]:
times_renewed1=times_renewed.groupBy('golden_id').agg(sum('times_renewed').alias("renewal_count"))

In [ ]:
# grp=['policy_number', 'certificate_no']
life_deatils=life_deatils.join(times_renewed1,'golden_id','left').drop_duplicates()

In [ ]:
life_deatils=life_deatils.join(tenure_df,'golden_id','left').drop_duplicates()

In [ ]:
active_pol_df=policy_summary.selectExpr('golden_id','status','policy_no as policy_number','certificate_no')

cols=['golden_id','status']
res_status_cnt=active_pol_df.groupBy(cols).agg(countDistinct(when(life_deatils['status']=='Active',concat_ws("|",col('policy_number'),col('certificate_no')))).alias('active_policies_t'),
                                         countDistinct(when(life_deatils['status']=='Inactive',concat_ws("|",col('policy_number'),col('certificate_no')))).alias('inactive_policies_t'))


In [ ]:
res_status_cnt=res_status_cnt.drop_duplicates()

In [ ]:
life_time_deatils=life_deatils.join(res_status_cnt,'golden_id','left').drop_duplicates()

In [ ]:
life_time_deatils=life_time_deatils.withColumn('claim_no',when(col('claim_no')=='-',lit(None)).otherwise(col('claim_no')))

In [ ]:
life_time_deatil1=life_time_deatils.groupBy('golden_id').agg(countDistinct('claim_no').alias("claim_paid_count"),
                                                       sum('total_claim_amount').alias("claim_paid"),
                                                       max('active_policies_t').alias("active_policies"),
                                                       max('inactive_policies_t').alias("inactive_policies"))

In [ ]:
life_time_deatil1

In [ ]:
life_time_deatils=life_time_deatils.join(life_time_deatil1,'golden_id','left').drop_duplicates()


In [ ]:
life_time_deatils.columns

In [ ]:
life_time_deatils=life_time_deatils.join(premium_temp,'golden_id','inner')#right

In [ ]:
life_time_deatils=life_time_deatils.withColumn('life_time_value',lit('-'))

In [ ]:
life_time_deatils=life_time_deatils.selectExpr('golden_id',"tenure","active_policies as activePolicies","inactive_policies as inactivePolicies","renewal_count as renewalCount","life_time_value as lifetimeValue","premium_received as premiumReceived","claim_paid as claimPaid","claim_paid_count as claimPaidCount")

In [ ]:
life_time_deatils=life_time_deatils.select('golden_id', col('tenure').cast('String'), 'activePolicies', 'inactivePolicies', 'renewalCount', 'lifetimeValue', 'premiumReceived', 'claimPaid', 'claimPaidCount')

In [ ]:
life_time_deatils=life_time_deatils.withColumn('tenure',round((col('tenure')/365),2)).withColumn('premiumReceived',round(col('premiumReceived'),2))\
.withColumn('claimPaid',round(col('claimPaid'),2))

In [ ]:
life_time_deatils=life_time_deatils.drop_duplicates()
life_time_deatils.persist()
life_time_deatils.count()

In [ ]:
output_index = 'life_time_deatils_v2'#cpa_all_prod_fy20,cpa_all_prod_fy21
schema = 'customermart'

import sys
try:
    
     life_time_deatils.write.option("truncate", "true").format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('overwrite').save()
    
    
except Exception as e:
    x = e
    print(x)
else:
    x = 'success'
    print(x)

In [ ]:
target_index='get_spa_cust_life_time_details'
print(target_index)

In [ ]:
import json, pprint,requests
es_nodes = '10.35.12.212'
es_port = '9200'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'

######target_index = 'actuarial_premium_risk_tables'----previous name...delete later

#alias_name = 'nepuepr_uat_3'
target_doc_type = 'get_spa_cust_life_time_details'
#########delete index if already exists########################################################################################################################################################################

In [ ]:
target_index_doc_type = target_index +'/'+ target_doc_type

In [ ]:
######### create mapping##################################################################################################################################################################################################################################################################
mapping_output = {target_doc_type:{"properties": {
"golden_id": {
"type": "keyword"
},
"tenure": {
"type": "keyword","index":"false"
},
"activePolicies": {
"type": "keyword","index":"false"
},
"inactivePolicies": {
"type":"keyword","index":"false"
},
"renewalCount": {
"type": "keyword","index":"false"
},
"lifetimeValue": {
"type": "keyword","index":"false"
},
"premiumReceived": {
"type": "keyword"
},
"claimPaid": {
"type": "keyword","index":"false"
},
"claimPaidCount": {
"type": "keyword","index":"false"
}}
}}



URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/'+target_index+'/_mapping/'+target_doc_type
headers = {'Content-Type': 'application/json'}
r = requests.put(URL, data=json.dumps(mapping_output), headers=headers)
print(r)

In [ ]:
try:
     life_time_deatils.write.format('org.elasticsearch.spark.sql').option('es.index.auto.create', 'true')\
        .option('es.nodes' , es_nodes).option('es.port', es_port).option('es.resource', target_index_doc_type)\
        .option("es.write.operation","upsert").option("es.mapping.id","golden_id").mode("append").save() 
except Exception as e :
    x = e
else: 
    x = 200 # success
print(x)

# Interaction_journal

In [ ]:
gscPythonOptions = {
         "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpcustcoe","password": "gpcustcoe@123",
    "dbschema":"customermart"
    ,"dbtable": "interaction_journal",
         "server.port":"1150-1170"}


ij_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.select("source_app", "interaction_type", "interaction_date", "policy_no", "certificate_no", "reference_num", 
"transaction_id", "ticket_no", "mobile", "email", "awb_no", "customer_name", "deposition", "sub_deposition", "callback_date", 
"remarks", "language_pref", "language_barrier", "lost_reason", "competitor_name", "ticket_close_date", "ticket_tat", "ticket_status", 
"ticket_source", "open_status", "link_status", "print_flag", "courier", "delivery_return_date", "recieved_by","source_system_customer_id")

In [ ]:
ij_gid=header_df.join(ij_df,'source_system_customer_id','inner')

In [ ]:
ij_gid.printSchema()

In [ ]:
ij_gid=ij_gid.withColumn('interaction_date',date_format(to_utc_timestamp(col('interaction_date'),"UTC"),'yyyy-MM-dd'))\
.withColumn('callback_date',date_format(to_utc_timestamp(col('callback_date'),"UTC"),'yyyy-MM-dd'))\
.withColumn('ticket_close_date',date_format(to_utc_timestamp(col('ticket_close_date'),"UTC"),'yyyy-MM-dd'))\
.withColumn('delivery_return_date',date_format(to_utc_timestamp(col('delivery_return_date'),"UTC"),'yyyy-MM-dd'))

In [ ]:
ij_gid=ij_gid.drop_duplicates()

In [ ]:
# ij_gid=ij_gid.join(cust_detail.select('golden_id'),'golden_id','inner')

In [ ]:
ij_gid.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
records=ij_gid.count()
records

In [ ]:
# target_index='get_spa_interaction_journal_summary'
# print(target_index)

In [ ]:
# import json, pprint, requests
# es_nodes = '10.35.12.212'
# es_port = '9200'
# es_user = 'elastic'
# es_pwd = 'bEiilauM3es'

# ######target_index = 'actuarial_premium_risk_tables'----previous name...delete later

# #alias_name = 'nepuepr_uat_3'
# target_doc_type = 'get_spa_interaction_journal_summary'
# #########delete index if already exists########################################################################################################################################################################

In [ ]:
# target_index_doc_type = target_index +'/'+ target_doc_type

In [ ]:
# try:
#      ij_gid.write.format('org.elasticsearch.spark.sql').option('es.index.auto.create', 'true')\
#         .option('es.nodes' , es_nodes).option('es.port', es_port).option('es.resource', target_index_doc_type)\
#         .option("es.write.operation","upsert").option("es.mapping.id",'golden_id').mode("append").save() 
# except Exception as e :
#     x = e
# else: 
#     x = 200 #success
# print(x)

In [ ]:
# # Interaction_journal

# gscPythonOptions = {
#          "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpcustcoe","password": "gpcustcoe@123",
#     "dbschema":"customermart"
#     ,"dbtable": "interaction_journal",
#          "server.port":"1150-1170"}


# ij_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
# .select("source_app", "interaction_type", "interaction_date", "policy_no", "certificate_no", "reference_num", 
# "transaction_id", "ticket_no", "mobile", "email", "awb_no", "customer_name", "deposition", "sub_deposition", "callback_date", 
# "remarks", "language_pref", "language_barrier", "lost_reason", "competitor_name", "ticket_close_date", "ticket_tat", "ticket_status", 
# "ticket_source", "open_status", "link_status", "print_flag", "courier", "delivery_return_date", "recieved_by","source_system_customer_id")

# ij_gid=header_df.join(ij_df,'source_system_customer_id','inner')

# ij_gid.printSchema()

# ij_gid=ij_gid.withColumn('interaction_date',date_format(to_utc_timestamp(col('interaction_date'),"UTC"),'yyyy-MM-dd'))\
# .withColumn('callback_date',date_format(to_utc_timestamp(col('callback_date'),"UTC"),'yyyy-MM-dd'))\
# .withColumn('ticket_close_date',date_format(to_utc_timestamp(col('ticket_close_date'),"UTC"),'yyyy-MM-dd'))\
# .withColumn('delivery_return_date',date_format(to_utc_timestamp(col('delivery_return_date'),"UTC"),'yyyy-MM-dd'))

# ij_gid=ij_gid.drop_duplicates()

# ij_gid=ij_gid.join(cust_detail.select('golden_id'),'golden_id','inner')

# ij_gid.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
# records=ij_gid.count()
# records

# target_index='get_spa_interaction_journal_summary'
# print(target_index)

# import json, pprint, requests
# es_nodes = '10.35.12.212'
# es_port = '9200'
# es_user = 'elastic'
# es_pwd = 'bEiilauM3es'

# ######target_index = 'actuarial_premium_risk_tables'----previous name...delete later

# #alias_name = 'nepuepr_uat_3'
# target_doc_type = 'get_spa_interaction_journal_summary'
# #########delete index if already exists########################################################################################################################################################################

# target_index_doc_type = target_index +'/'+ target_doc_type

# try:
#      ij_gid.write.format('org.elasticsearch.spark.sql').option('es.index.auto.create', 'true')\
#         .option('es.nodes' , es_nodes).option('es.port', es_port).option('es.resource', target_index_doc_type)\
#         .option("es.write.operation","upsert").option("es.mapping.id",'golden_id').mode("append").save() 
# except Exception as e :
#     x = e
# else: 
#     x = 200 #success
# print(x)

In [ ]:
import io
import pandas as pd
import xlsxwriter
import sys
import smtplib
from os.path import basename
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate
import pandas as pd
from sqlalchemy import create_engine
from smtplib import SMTP

if x!='success':
    status='FAILED'
else:
    status="SUCCESS"

subject=f"Automated Mail-SPA PART-3 Job Status : {status}"

if x=="SUCCESS":
    recipients = ['nishant5.addenki@tataaig.com','devnandan.TataIndustries@tataaig.com','mayur.TataIndustries@tataaig.com','Ajeet3.Jaiswal@tataaig.com'] 
    emaillist = [elem.strip().split(',') for elem in recipients]
    msg = MIMEMultipart()
    msg['Subject'] = subject
    msg['From'] = 'customercoe@tataaig.com'


    html = """\
    <html>
      <head></head>
      <body>
        SPA PART-3 job status is: Successful
      </body>
        <p>This is automated mail, please don't reply. Thanks</p>
    </html>
    """

    part1 = MIMEText(html, 'html')
    msg.attach(part1)

    server = smtplib.SMTP("tataaig-com.mail.protection.outlook.com",25)
    server.sendmail(msg['From'], emaillist , msg.as_string())
else:
      
    recipients = ['nishant5.addenki@tataaig.com','devnandan.TataIndustries@tataaig.com','mayur.TataIndustries@tataaig.com','Ajeet3.Jaiswal@tataaig.com'] 
    emaillist = [elem.strip().split(',') for elem in recipients]
    msg = MIMEMultipart()
    msg['Subject'] = subject
    msg['From'] = 'customercoe@tataaig.com'


    html = """\
    <html>
      <head></head>
      <body>
        CPA job status is: Failed
      </body>
        <p>This is automated mail, please don't reply. Thanks</p>
    </html>
    """

    part1 = MIMEText(html, 'html')
    msg.attach(part1)

    server = smtplib.SMTP("tataaig-com.mail.protection.outlook.com",25)
    server.sendmail(msg['From'], emaillist , msg.as_string())

In [ ]:
spark.stop()